# MSEE UQ short course:  The $\texttt{UQpy}$ library

Application of forward UQ using the $\texttt{UQpy}$ modules $\texttt{run}\_\texttt{model}, \texttt{distributions}$ and $\texttt{sampling}$.

Detailed instructions on how to use these modules can be found in the $\texttt{UQpy}$ documentation.


https://uqpyproject.readthedocs.io/en/latest/runmodel_doc.html

https://uqpyproject.readthedocs.io/en/latest/distributions/index.html

https://uqpyproject.readthedocs.io/en/latest/sampling/index.html

In [1]:
import numpy as np
from model import boucwen

# Exercise 1 


## Linking a $\texttt{Python}$ computational model with $\texttt{UQpy}$

The model consists in a highly nonlinear single degree of freedom system represented by a Bouc-Wen model of hysteresis:

   \begin{align*}
    & m \ddot{z}(t) + k r(t) = - \ddot{u}(t) \\
    & \dot{r}(t) = \dot{z} - \beta \vert\dot{z}(t)\vert \vert r(t) \vert^{2} r(t) - \gamma \dot{z}(t) \vert r(t) \vert^{3}
   \end{align*}

where $\ddot{u}(t)$ is the ground motion exciting the system. The Bouc-Wen model of hysteresis is parameterized by stiffness $k$, parameter $r_{0}=\sqrt[3]{\frac{1}{\beta-\gamma}}$ and parameter $\delta=\frac{\beta}{\beta+\gamma}$. Forward simulation of the problem is performed via a 4th-order Runge-Kutta method. 
This example makes use of the following file:
- $\texttt{USACA47.035.txt}$ that contains the El-Centro earthquake ground motion time-series, downloaded from the Center for Engineering Strong Motion Data.

# Part 1

You are provided with the $\texttt{boucwen}$ function in the script $\texttt{model.py}$. This function runs the Bouc-Wen model of hysterisis for the El-centro earthquake ground motion excitation. This funtion returns the maximum displacement $z(t)$ of the system.

Run the model for the nominal set of parameters $k=1.0$, $r_{0}=2.5$, $\delta=0.9$, and print the output of the function.

In [2]:
##Solution
from model import *

max_disp=boucwen(np.atleast_2d([1.0, 2.5, 0.9]))


# Part 2

Run the Bouc-Wen model for the same set of parameters, using the $\texttt{RunModel}$ class of $\texttt{UQpy}$. 

### Step 1

Import $\texttt{RunModel}$ class from $\texttt{UQpy.run}$\_$\texttt{model}$ module.

Since the computational model to be executed is a Python model, we also need to import $\texttt{PythonModel}$ from the same module.

In [3]:
# Solution
from UQpy.run_model.RunModel import RunModel
from UQpy.run_model.model_execution.PythonModel import PythonModel

### Step 2

Instantiate a $\texttt{PythonModel}$ object:
- Define $\texttt{model.py}$ as the $\texttt{model}$\_$\texttt{script}$ attribute.
- Define $\texttt{boucwen}$ as the $\texttt{model}$\_$\texttt{object}\_\texttt{name}$ attribute.

Instantiate a $\texttt{RunModel}$ object: 
- Define the $\texttt{PythonModel}$ object created above as the $\texttt{model}$ attribute.



In [4]:
#Solution
from UQpy.sampling import MonteCarloSampling
from UQpy.distributions import Normal, Uniform, Lognormal, JointIndependent

##Run the model
m=PythonModel(model_script='model.py', model_object_name='boucwen', var_names=['k', 'r0', 'delta'])
boucwen = RunModel(model=m)

### Step 3

Generate an ndarray containing the set of parameters (the shape of the array should be (1, 3)).



In [5]:
# Solution
import numpy as np
samples=np.array([1.0, 2.5, 0.9]).reshape(1,-1)

### Step 4

Run the model for this set of parameters using the $\texttt{run}$ method of the $\texttt{RunModel}$ object. Print the response (i.e.,  $\texttt{qoi}$\_$\texttt{list}$ attribute of the  $\texttt{RunModel}$ object).

In [6]:
boucwen.run(samples=samples)
print(boucwen.qoi_list)

[5.029733072011507]


# Exercise 2 


## Linking a third-party software computational model with $\texttt{UQpy}$


The model consists of an identation test performed on a cuboid sample with the aid of an elastic contact sphere. 


<img src="IndentationTest.png" width="500"> 


The example is adopted from  https://sfepy.org/doc-devel/examples/linear_elasticity-elastic_contact_sphere.html. 
The analysis is performed using the Python package $\texttt{sfepy}$. Visualization can be performed with Paraview. Even though the problem is composed by linear materials and described by small deformations, it is highly non-linear due to the contact boundary conditions between the sphere and the cube. 

The system is parameterized by two parameters: (i) the elastic sphere stiffness parameter $\texttt{k}$ and, (ii) the force $\texttt{f0}$ at the contact point before identation begins. The response of the model consists of the maximum absolute value of the displacement field at the identation point. 

# Part 1

You are provided with the following scripts:

1. $\texttt{elastic}$\_$\texttt{contact}$\_$\texttt{sphere.py}$  script which runs the contact sphere model model.
2. $\texttt{simple.py}$ script that contains the $\texttt{sfepy}$ solver that executes the identation test.

$\texttt{simple.py}$ script and the $\texttt{elastic}$\_$\texttt{contact}$\_$\texttt{sphere.py}$ script must be in the same directory in order for the solver to be executed.

- Run the elastic sphere contact sphere model for the nominal parameters $\texttt{k}=10^5$, $\texttt{f0}=0.01$. You can type the following command in a cell:

    - $\texttt{%run}$  $\texttt{simple.py}$ $\texttt{elastic}$_ $\texttt{contact}$_ $\texttt{sphere.py}$. This will create the output file  
    $\texttt{cube}$_ $\texttt{medium}$_$\texttt{hexa.vtk}$ that contains the displacement field. 
    
    
- Read the output file by typing the following commands in a cell:

    - $\texttt{import meshio}$
    - $\texttt{mesh=meshio.read('cube}$\_$\texttt{medium}$\_$\texttt{hexa.vtk'})$
    - $\texttt{disp=mesh.point}$\_$\texttt{data['u']}$
    - $\texttt{y =max(abs(disp[:, 2])))}$
    
- Print the response

In [1]:
# Solution
%run simple.py elastic_contact_sphere_initial.py

import meshio
mesh=meshio.read('cube_medium_hexa.vtk')
disp=mesh.point_data['u']
y=max(abs(disp[:,2]))

print(y)

sfepy: left over: ['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__file__', '__cached__', '__builtins__', 'absolute_import', 'data_dir', 'stiffness_from_lame', 'k', 'f0', 'main', 'verbose', '_filename']
sfepy: reading mesh (/Users/dimitris/Desktop/Day1/Day1_InClassExercises_Solutions/cube_medium_hexa.mesh)...
sfepy:   number of vertices: 1331
sfepy:   number of cells:
sfepy:     3_8: 1000
sfepy: ...done in 0.01 s
sfepy: creating regions...
sfepy:     Omega
sfepy:     Bottom
sfepy:     Top
sfepy: ...done in 0.01 s
sfepy: equation "elasticity":
sfepy: dw_lin_elastic.2.Omega(solid.D, v, u)
     + dw_contact_sphere.2.Top(cs.f, cs.c, cs.r, v, u)
     = 0
sfepy: using solvers:
                ts: no ts
               nls: newton
                ls: ls
sfepy: updating variables...
sfepy: ...done
sfepy: setting up dof connectivities...
sfepy: ...done in 0.00 s
sfepy: matrix shape: (3630, 3630)
sfepy: assembling matrix graph...
sfepy: ...done in 0.02 s
sfepy: matrix structura

# Part 2

Run the contact sphere model for the nominal set of parameters using the $\texttt{RunModel}$ of $\texttt{UQpy}$. You are provided with the additional files:

1. $\texttt{PythonAsThirdParty.py}$ 
2. $\texttt{process}$\_$\texttt{3rd}$\_$\texttt{party}$\_$\texttt{output.py}$ 



### Step 1

File $\texttt{elastic}$\_$\texttt{contact}$\_$\texttt{sphere.py}$ will serve as the $\texttt{input}$\_$\texttt{template}$ file in this example. Your first task is to modify it to accept values for the parameters $\texttt{k}$ and $\texttt{f0}$: Replace their values with angle brackets $<,>$ with the variable names inside. The variable names are provided in the $\texttt{var}$\_$\texttt{names}$ list, i.e., $\texttt{<stifness>}$ and $\texttt{<force>}$.


### Step 2

File $\texttt{PythonAsThirdParty.py}$ serves as $\texttt{model}$\_$\texttt{script}$. This file contains the function $\texttt{run}$\_$\texttt{model}$ that runs the $\texttt{Python}$ $\texttt{sfepy}$ model as a third-party software. You have to complete this function  to perform the following operations:

- The function $\texttt{run}$\_$\texttt{model}$ should take a single input argument $\texttt{n}$ (the python index of the run). The function should not return anything. 

- Import  $\texttt{Python}$ library  $\texttt{os}$. 

- Copy the $\texttt{input}$\_$\texttt{template}$ script that corresponds to run $\texttt{n}$,  (i.e.,$\texttt{elastic}$\_$\texttt{contact}$\_$\texttt{sphere}$\_$\texttt{n.py'}$) from  directory $\texttt{../Model}$\_$\texttt{Runs/run}$\_$\texttt{n/InputFiles/}$ to the current working directory  $\texttt{../Model}$\_$\texttt{Runs/run}$\_$\texttt{n/}$.

- Run the model using  $\texttt{os.system(command})$ where

$\texttt{command = 'python3 simple.py}$ 
$\texttt{elastic}$\_$\texttt{contact}$\_$\texttt{sphere}$\_$\texttt{n.py'}$.

- Copy the generated file $\texttt{cube}$\_$\texttt{medium}$\_$\texttt{hexa}$\_$\texttt{index.vtk}$ from the current working directory  $\texttt{../Model}$\_$\texttt{Runs/run}$\_$\texttt{n/}$ to $\texttt{../Model}$\_$\texttt{Runs/run}$\_$\texttt{n/OutputFiles/}$ directory. 


### Step 3

File $\texttt{process}$\_$\texttt{3rd}$\_$\texttt{party}$\_$\texttt{output.py}$ serves as  $\texttt{output}$\_$\texttt{script}$. Complete function $\texttt{read}$\_$\texttt{output}$ to take a single input argument  $\texttt{n}$, that is the python index of the run. The function should reads the generated $\texttt{.vtk}$ file and returns the response.


### Step 4

- Instantiate a $\texttt{RunModel}$ object: 
- Create a $\texttt{ThirdPartyModel}$ object that contains all the third-party model files.
- Generate an ndarray containing the set of parameters (the shape of the array should be (1, 2)).
- Run the model for this set of parameters using the $\texttt{run}$ method of the $\texttt{RunModel}$ object.
- Print the response (i.e.,  $\texttt{qoi}$\_$\texttt{list}$ attribute of the  $\texttt{RunModel}$ object).

In [3]:
#Solution
import numpy as np
import matplotlib.pyplot as plt
from UQpy.run_model.RunModel import RunModel
from UQpy.run_model.model_execution.ThirdPartyModel import ThirdPartyModel

samples=np.array([1e4,2e-2]).reshape(1,-1)

m1=ThirdPartyModel(model_script='PythonAsThirdParty_model.py', model_object_name = 'run_model',
                  input_template='elastic_contact_sphere.py', var_names=['k', 'f0'],
                  output_script='process_3rd_party_output.py', output_object_name='read_output')
model_serial_third_party=RunModel(samples=samples,  model=m1)

print(model_serial_third_party.qoi_list)

/Users/dimitris/Desktop/Day1/Day1_InClassExercises_Solutions/Model_Runs_2022_05_25_11_51_053517/run_0/InputFiles/elastic_contact_sphere_0.py
/Users/dimitris/Desktop/Day1/Day1_InClassExercises_Solutions/Model_Runs_2022_05_25_11_51_053517/run_0/elastic_contact_sphere_0.py
cp -r "/Users/dimitris/Desktop/Day1/Day1_InClassExercises_Solutions/Model_Runs_2022_05_25_11_51_053517/run_0/InputFiles/elastic_contact_sphere_0.py" "/Users/dimitris/Desktop/Day1/Day1_InClassExercises_Solutions/Model_Runs_2022_05_25_11_51_053517/run_0/elastic_contact_sphere_0.py"
python3 simple.py elastic_contact_sphere_0.py
sfepy: left over: ['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__file__', '__cached__', '__builtins__', 'absolute_import', 'data_dir', 'stiffness_from_lame', 'k', 'f0', 'main', 'verbose', '_filename']
sfepy: reading mesh (/Users/dimitris/Desktop/Day1/Day1_InClassExercises_Solutions/Model_Runs_2022_05_25_11_51_053517/run_0/cube_medium_hexa.mesh)...
sfepy:   number of vertices: 13

# Exercise 3 


## Forward propagation of uncertainties using $\texttt{UQpy}$

In this exercise you have to propagate uncertainties through the Bouc-Wen computational model. Randomness is assumed in the systems' parameters [$k, r_{0}, \delta$]. The parameters are considered to be independent, with the following probability distribution models:

- Parameter $k$ is assumed to be uniformly distributed, i.e., $k \sim \mathcal{U}(0.5, 2.5)$. 
- Parameter $r_{0}$ is assumed to be normally distributed, i.e., $r_0 \sim \mathcal{N}(2.5, 0.01)$. 
- Parameter $\delta$ is assumed to be normally distributed, i.e.,  $\delta \sim\mathcal{N}(0.9, 0.04)$.

Realizations of the input uncertain parameters are obtained via Monte Carlo sampling.

## Part 1

Create a distribution object for each parameter (random variable).

### Step 1

Import the necessary probability models i.e., $\texttt{Uniform}$ and $\texttt{Normal}$  from the $\texttt{distributions}$ module of $\texttt{UQpy}$.

In [ ]:
# Solution
from UQpy.distributions import Uniform, Normal

### Step 2

For each random variable create a $\texttt{Distribution}$ object. Define a joint distribution object from the independent distributions. To this end you will also need: 

- Import $\texttt{JointIndependent}$ class from $\texttt{distributions}$ module of $\texttt{UQpy}$.

- Create a $\texttt{JointIndependent}$  object by providing a list of the marginal distributions.

In [ ]:
#Solution
from UQpy.distributions import JointIndependent
dist1=Uniform(0.5,2.0)
dist2=Normal(2.5, 0.01)
dist3=Normal(0.9, 0.04)
joint_distribution=JointIndependent(marginals=[dist1,dist2,dist3])

## Part 2

Create a $\texttt{MonteCarloSampling}$ object.

- Import  $\texttt{MonteCarloSampling}$ class from the $\texttt{sampling}$ module of $\texttt{UQpy}$.

- Create $\texttt{MonteCarloSampling}$ object, i.e., provide the $\texttt{JointIndependent}$ as the $\texttt{distributions}$ parameter and define the number of samples: $\texttt{nsamples}=10$.

In [ ]:
#Solution
from UQpy.sampling import MonteCarloSampling
x = MonteCarloSampling(distributions=joint_distribution, nsamples=10)
samples = x.samples

## Part 3

Forward propagation of the undertainties through the Bouc-Wen computational model.

- Import $\texttt{RunModel}$ from $\texttt{UQpy.run}$\_$\texttt{model}$ module.
- Instantiate a $\texttt{PythonModel}$ object that serves as an input to the $\texttt{RunModel}$ object
- Run the model for the sets of parameters using the $\texttt{run}$ method of the $\texttt{RunModel}$ object.
- Plot the histogram of the response.

In [ ]:
#Solution
from UQpy.run_model.RunModel import RunModel

##Run the model
m2=PythonModel(model_script='model.py', model_object_name='boucwen_runmodel', 
               var_names=['k', 'r0', 'delta'])
boucwen = RunModel(model=m2)
boucwen.run(samples=samples)
qoi = boucwen.qoi_list


import matplotlib.pyplot as plt

# An "interface" to matplotlib.axes.Axes.hist() method
n, bins, patches = plt.hist(x=boucwen.qoi_list, bins='auto')
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Response')
plt.ylabel('Frequency')